In [8]:
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
import os
import pandas as pd

# Path ke folder yang berisi file-file DataFrame
folder_path = '../comodity-price-prediction-penyisihan-arkavidia-9/Harga Bahan Pangan/train'

# Dictionary untuk menyimpan semua DataFrame dengan nama file sebagai key
df_dict = {}

# Loop melalui semua file dalam folder
for file_name in os.listdir(folder_path):
    # Pastikan file yang dibaca adalah file yang diinginkan (misalnya, CSV)
    if file_name.endswith('.csv'):
        # Buat path lengkap ke file
        file_path = os.path.join(folder_path, file_name)
        
        # Baca file dan simpan ke dictionary dengan nama file sebagai key
        df = pd.read_csv(file_path)
        df_dict[file_name] = df  # Gunakan nama file sebagai key


In [10]:
df_bawang_merah = df_dict['Bawang Merah.csv']
df_bawang_putih = df_dict['Bawang Putih Bonggol.csv']
df_beras_medium = df_dict['Beras Medium.csv']
df_beras_premium = df_dict['Beras Premium.csv']
df_cabai_merah_keriting = df_dict['Cabai Merah Keriting.csv']
df_cabai_rawit_merah = df_dict['Cabai Rawit Merah.csv']
df_daging_ayam = df_dict['Daging Ayam Ras.csv']
df_daging_sapi = df_dict['Daging Sapi Murni.csv']
df_gula = df_dict['Gula Konsumsi.csv']
df_minyak_goreng_curah = df_dict['Minyak Goreng Curah.csv']
df_minyak_goreng_kemasan = df_dict['Minyak Goreng Kemasan Sederhana.csv']
df_telur_ayam = df_dict['Telur Ayam Ras.csv']
df_tepung_terigu = df_dict['Tepung Terigu (Curah).csv']

In [11]:
import pandas as pd

# List semua dataframe
df_list = [
    df_bawang_merah, df_bawang_putih, df_beras_medium, df_beras_premium, df_cabai_merah_keriting,
    df_cabai_rawit_merah, df_daging_ayam, df_daging_sapi, df_gula, df_minyak_goreng_curah,
    df_minyak_goreng_kemasan, df_telur_ayam, df_tepung_terigu
]

# Loop setiap dataframe dan lakukan interpolasi
for i in range(len(df_list)):
    df = df_list[i]  # Ambil dataframe

    # Pisahkan kolom tanggal (asumsi ada kolom 'date' atau 'tanggal')
    date_cols = df.select_dtypes(include=['datetime64', 'object'])  # Deteksi kolom tanggal
    num_cols = df.select_dtypes(include=['number'])  # Ambil hanya kolom numerik

    # Interpolasi hanya untuk kolom numerik
    df_num_imputed = num_cols.interpolate(method='linear', limit_direction='both')

    # Gabungkan kembali kolom tanggal dengan hasil interpolasi
    df_list[i] = pd.concat([date_cols.reset_index(drop=True), df_num_imputed.reset_index(drop=True)], axis=1)


In [12]:
df_dict = {
    'Bawang Merah': df_list[0],
    'Bawang Putih Bonggol': df_list[1],
    'Beras Medium': df_list[2],
    'Beras Premium': df_list[3],
    'Cabai Merah Keriting': df_list[4],
    'Cabai Rawit Merah': df_list[5],
    'Daging Ayam Ras': df_list[6],
    'Daging Sapi Murni': df_list[7],
    'Gula Konsumsi': df_list[8],
    'Minyak Goreng Curah': df_list[9],
    'Minyak Goreng Kemasan Sederhana': df_list[10],
    'Telur Ayam Ras': df_list[11],
    'Tepung Terigu (Curah)': df_list[12]
}


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Parameter
seq_length = 30  # Panjang sequence
learning_rate = 0.001
epochs = 100
# Rentang waktu prediksi
predict_start = '2024-10-01'
predict_end = '2024-12-31'
predict_periods = (pd.to_datetime(predict_end) - pd.to_datetime(predict_start)).days + 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [14]:
class BiGRUPredictor(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, num_layers=3, output_size=1, dropout=0.3):
        super(BiGRUPredictor, self).__init__()

        # Bi-GRU Layer
        self.gru = nn.GRU(input_size, hidden_size, num_layers, 
                          batch_first=True, dropout=dropout, bidirectional=True)

        # Fully Connected Layer
        self.fc = nn.Linear(hidden_size * 2, output_size)  # *2 karena bidirectional

    def forward(self, x):
        gru_out, _ = self.gru(x)  # (batch, seq_len, hidden_size * 2)
        out = self.fc(gru_out[:, -1, :])  # Ambil output dari timestep terakhir
        return out.unsqueeze(-1)  # Pastikan output (batch_size, 1, 1)

In [15]:
# Direktori penyimpanan model terbaik
if not os.path.exists("best_models_bigru"): os.makedirs("best_models_bigru")

# Menyimpan hasil prediksi dan MAPE terbaik
all_predictions = []
mape_results = {}

In [16]:
for komoditas, df_komoditas in df_dict.items():
    print(f"\n=== Memproses {komoditas} ===")

    df_komoditas['Date'] = pd.to_datetime(df_komoditas['Date'])
    
    for provinsi in df_komoditas.columns[1:]:
        print(f"  → Prediksi untuk {provinsi}...")

        df_prov = df_komoditas[['Date', provinsi]].rename(columns={'Date': 'ds', provinsi: 'y'})
        
        # Normalisasi data
        scaler = MinMaxScaler()
        df_scaled = scaler.fit_transform(df_prov[['y']])
        
        X, y = [], []
        for i in range(len(df_scaled) - seq_length):
            X.append(df_scaled[i:i+seq_length])  # Menggunakan hanya harga komoditas sebagai input
            y.append(df_scaled[i+seq_length])  # Target harga komoditas
        
        X, y = np.array(X), np.array(y)
        X = torch.tensor(X, dtype=torch.float32).to(device)
        y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1).to(device)
        
        # Pisahkan training & testing
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        # Bangun model LSTM
        model = BiGRUPredictor(input_size=1, hidden_size=128, num_layers=3, output_size=1, dropout=0.3).to(device)
        criterion = nn.SmoothL1Loss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
        
        best_val_loss = float("inf")
        best_mape = float("inf")
        best_model_path = f"best_models/univariate_{komoditas}_{provinsi}.pt"

        # Train model
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            output = model(X_train)
            loss = criterion(output, y_train)
            loss.backward()
            optimizer.step()
            
            if (epoch + 1) % 10 == 0:
                model.eval()
                with torch.no_grad():
                    test_output = model(X_test)
                    val_loss = criterion(test_output, y_test).item()
                    
                    test_output_np = test_output.cpu().numpy().reshape(-1, 1)
                    y_test_np = y_test.cpu().numpy().reshape(-1, 1)

                    test_output_np = scaler.inverse_transform(test_output_np)
                    y_test_np = scaler.inverse_transform(y_test_np)

                    mape = mean_absolute_percentage_error(y_test_np, test_output_np)


                print(f"Epoch {epoch+1}/{epochs}, Training Loss: {loss.item():.6f}, Validation Loss: {val_loss:.6f}, MAPE: {mape:.6f}")
                
                scheduler.step(val_loss)
                
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_mape = mape
                    torch.save(model.state_dict(), best_model_path)
        
        mape_results[f"{komoditas}_{provinsi}"] = best_mape

        # Load model terbaik untuk prediksi masa depan
        best_model = BiGRUPredictor(input_size=1).to(device)
        best_model.load_state_dict(torch.load(best_model_path))
        best_model.eval()
        
        last_sequence = X[-1].unsqueeze(0)
        future_preds = []

        for _ in range(predict_periods):
            with torch.no_grad():
                next_pred = best_model(last_sequence).item()
            future_preds.append(next_pred)
            
            next_input = torch.tensor([[next_pred]], dtype=torch.float32).unsqueeze(0).to(device)
            last_sequence = torch.cat([last_sequence[:, 1:, :], next_input], dim=1)

        future_preds = np.array(future_preds).reshape(-1, 1)
        future_preds = scaler.inverse_transform(future_preds)

        for i, pred in enumerate(future_preds):
            date = pd.to_datetime(predict_start) + pd.Timedelta(days=i)
            all_predictions.append({
                'id': f"{komoditas}/{provinsi}/{date.strftime('%Y-%m-%d')}",
                'price': pred[0]
            })

# Simpan hasil prediksi ke CSV
submission_df = pd.DataFrame(all_predictions)
submission_df.to_csv('submission_univariate_bi_gru.csv', index=False)


=== Memproses Bawang Merah ===
  → Prediksi untuk Aceh...


/home/sirius/Documents/DS/myenv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 10/100, Training Loss: 0.015596, Validation Loss: 0.056975, MAPE: 0.221919
Epoch 20/100, Training Loss: 0.004819, Validation Loss: 0.018387, MAPE: 0.144573
Epoch 30/100, Training Loss: 0.001009, Validation Loss: 0.001906, MAPE: 0.051476
Epoch 40/100, Training Loss: 0.000634, Validation Loss: 0.002065, MAPE: 0.037513
Epoch 50/100, Training Loss: 0.000628, Validation Loss: 0.001822, MAPE: 0.036782
Epoch 60/100, Training Loss: 0.000537, Validation Loss: 0.001084, MAPE: 0.031904
Epoch 70/100, Training Loss: 0.000504, Validation Loss: 0.001211, MAPE: 0.030388
Epoch 80/100, Training Loss: 0.000474, Validation Loss: 0.001059, MAPE: 0.028578
Epoch 90/100, Training Loss: 0.000475, Validation Loss: 0.001044, MAPE: 0.029068
Epoch 100/100, Training Loss: 0.000495, Validation Loss: 0.001047, MAPE: 0.029017
  → Prediksi untuk Bali...


/tmp/ipykernel_255370/1031921258.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv

Epoch 10/100, Training Loss: 0.015226, Validation Loss: 0.032470, MAPE: 0.184587
Epoch 20/100, Training Loss: 0.008701, Validation Loss: 0.010479, MAPE: 0.168153
Epoch 30/100, Training Loss: 0.001544, Validation Loss: 0.002090, MAPE: 0.081154
Epoch 40/100, Training Loss: 0.001361, Validation Loss: 0.001622, MAPE: 0.054644
Epoch 50/100, Training Loss: 0.000949, Validation Loss: 0.001198, MAPE: 0.054323
Epoch 60/100, Training Loss: 0.000619, Validation Loss: 0.000815, MAPE: 0.038171
Epoch 70/100, Training Loss: 0.000583, Validation Loss: 0.000724, MAPE: 0.034163
Epoch 80/100, Training Loss: 0.000567, Validation Loss: 0.000730, MAPE: 0.033247
Epoch 90/100, Training Loss: 0.000540, Validation Loss: 0.000673, MAPE: 0.031346
Epoch 100/100, Training Loss: 0.000557, Validation Loss: 0.000679, MAPE: 0.031219
  → Prediksi untuk Banten...


...

/tmp/ipykernel_255370/1031921258.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv

Epoch 10/100, Training Loss: 0.017179, Validation Loss: 0.003143, MAPE: 0.023683
Epoch 20/100, Training Loss: 0.015360, Validation Loss: 0.041207, MAPE: 0.096144
Epoch 30/100, Training Loss: 0.006201, Validation Loss: 0.007912, MAPE: 0.041058
Epoch 40/100, Training Loss: 0.002220, Validation Loss: 0.000917, MAPE: 0.013000
Epoch 50/100, Training Loss: 0.000698, Validation Loss: 0.000605, MAPE: 0.010195
Epoch 60/100, Training Loss: 0.000622, Validation Loss: 0.000158, MAPE: 0.003617
Epoch 70/100, Training Loss: 0.000653, Validation Loss: 0.000393, MAPE: 0.008237
Epoch 80/100, Training Loss: 0.000564, Validation Loss: 0.000259, MAPE: 0.006040
Epoch 90/100, Training Loss: 0.000555, Validation Loss: 0.000201, MAPE: 0.004676
Epoch 100/100, Training Loss: 0.000546, Validation Loss: 0.000270, MAPE: 0.006216
  → Prediksi untuk Sumatera Utara...


/tmp/ipykernel_255370/1031921258.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv

Epoch 10/100, Training Loss: 0.025016, Validation Loss: 0.037381, MAPE: 0.074661
Epoch 20/100, Training Loss: 0.006865, Validation Loss: 0.002063, MAPE: 0.013697
Epoch 30/100, Training Loss: 0.002841, Validation Loss: 0.003094, MAPE: 0.019400
Epoch 40/100, Training Loss: 0.001131, Validation Loss: 0.001249, MAPE: 0.010659
Epoch 50/100, Training Loss: 0.000909, Validation Loss: 0.001096, MAPE: 0.009583
Epoch 60/100, Training Loss: 0.000624, Validation Loss: 0.001191, MAPE: 0.010196
Epoch 70/100, Training Loss: 0.000607, Validation Loss: 0.001006, MAPE: 0.008701
Epoch 80/100, Training Loss: 0.000533, Validation Loss: 0.001007, MAPE: 0.008693
Epoch 90/100, Training Loss: 0.000529, Validation Loss: 0.001010, MAPE: 0.008721
Epoch 100/100, Training Loss: 0.000600, Validation Loss: 0.000996, MAPE: 0.008510


/tmp/ipykernel_255370/1031921258.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))


In [17]:
avg_mape = np.mean(list(mape_results.values()))
print(f"\nRata-rata MAPE dari model terbaik: {avg_mape:.6f}")


Rata-rata MAPE dari model terbaik: 0.016070
